In [1]:
import pandas as pd
import numpy as np
import os
import re

BASE_DIR = os.getcwd()
DIR_NAME = os.path.basename(BASE_DIR)
print(BASE_DIR, DIR_NAME)
while DIR_NAME != "RakutenTeam": 
    BASE_DIR = os.path.realpath(os.path.join(os.path.dirname(__name__), '..'))
    DIR_NAME = os.path.basename(BASE_DIR)
    os.chdir(BASE_DIR)
    DATA_DIR = os.path.join(BASE_DIR, 'data')
    print( BASE_DIR, DATA_DIR, DIR_NAME)


c:\Users\mangg\projects\RakutenTeam\notebook\tests tests
C:\Users\mangg\projects\RakutenTeam\notebook C:\Users\mangg\projects\RakutenTeam\notebook\data notebook
C:\Users\mangg\projects\RakutenTeam C:\Users\mangg\projects\RakutenTeam\data RakutenTeam


In [2]:
from src.data_loader import import_data

folder_path = os.path.join(DATA_DIR, "raw")
print(folder_path)

C:\Users\mangg\projects\RakutenTeam\data\raw


In [3]:
df = import_data(folder_path=folder_path)

In [4]:
designations = df.designation
descriptions = df.description

In [5]:
from src.features.text.pipelines.cleaner import CleanTextPipeline

pipe = CleanTextPipeline()
cleaned_descriptions = pipe.fit_transform(descriptions)

C:\Users\mangg\projects\RakutenTeam\src\features\text\transformers\cleaners.py:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
C:\Users\mangg\projects\RakutenTeam\src\features\text\transformers\cleaners.py:24: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
C:\Users\mangg\projects\RakutenTeam\src\features\text\transformers\cleaners.py:47: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'lxml')
C:\Users\mangg\projects\RakutenTeam\src\features\text\transformers\cleaners.py:47: MarkupResemblesLocatorWarn

In [6]:

pattern1 = r'([?¿º¢©́])\1'
pattern2 = r'[å¿]'
pattern3 = r"[?\s:;\/\!]{3,}"
pattern4 = r'\w*[å¿]\w*'


In [7]:
cleaned_descriptions.str.count(pattern1).sum(), cleaned_descriptions.str.count(pattern2).sum(), cleaned_descriptions.str.count(pattern3).sum(), cleaned_descriptions.str.count(pattern4).sum()

(8487.0, 13180.0, 132837.0, 13180.0)

In [8]:
from src.features.text.pipelines.corrector import CleanEncodingPipeline

encoding_pipeline = CleanEncodingPipeline()

corrected_descriptions = encoding_pipeline.fit_transform(cleaned_descriptions)

Neither PyTorch nor TensorFlow >= 2.0 have been found.Models won't be available and only tokenizers, configurationand file/data utilities can be used.


Badword: roulettes¿une
 Text to correct: ce produit appartient à la collection jeux de la marque djeco. référence produit . 08466 a propos de la marque . djeco parce que le jeu participe de manière essentielle au développement de l'enfant djeco crée une collection toujours plus ludique éducative et imaginative. fidèle à sa démarche de création artistique djeco fait appel à des illustrateurs talentueux pour enrichir sa collection de jeux et de jouets. depuis sa création la société de jouets djeco accorde une place prépondérante à l'illustration. ainsi est née l'idée de proposer à des illustrateurs d'imaginer la chambre d'enfant de leur rêve et de créer des objets qui suscitent l'imaginaire. grâce à ces créations ludiques et poétiques les enfants mettent en scène leur propre monde. les petits garçons rêveront de partager les aventures extravagantes du chevalier fantastique et de ses drôles de compagnons de route. les petites filles vont se lover avec ravissement dans le royaume féerique 

In [9]:
corrected_descriptions.str.count(pattern1).sum(), corrected_descriptions.str.count(pattern2).sum(), corrected_descriptions.str.count(pattern3).sum(),  corrected_descriptions.str.count(pattern4).sum()


(0.0, 0.0, 1.0, 0.0)

In [10]:
k = 5
selected_filter = corrected_descriptions.str.count(pattern2) > 0

In [ ]:
phrase

In [12]:
accent_pattern = r"ã"
accent_pattern_filter = corrected_descriptions.str.count(accent_pattern) > 0
corrected_descriptions[accent_pattern_filter].count()
#corrected_descriptions.str.findall(accent_pattern)

25

In [ ]:
from src.features.text.transformers.encoding_corrector import BadWordsDetector, BadWordsCorrector, PySpellCorrector

corrector = PySpellCorrector()
corrected_text = corrector.fit_transform(corrected_descriptions)

In [ ]:
pattern_filter = corrected_descriptions.str.count(search_pattern) > 0
#bad_words[pattern_filter]
#new_df = pd.DataFrame({"description": corrected_descriptions, "bad_words": bad_words})

In [ ]:
corrected_text[pattern_filter]

In [ ]:
corrected_text.str.count(pattern1).sum(), corrected_text.str.count(pattern2).sum(), corrected_text.str.count(pattern3).sum(),  corrected_text.str.count(pattern4).sum()


In [ ]:
corrected_text

In [ ]:
last_pattern = r"\w*[©́]+"
#corrected_text.str.findall(last_pattern)
last_pattern_filter =  corrected_text.str.count(last_pattern) > 0
corrected_text.str.count(last_pattern).sum()

In [ ]:
indexes = corrected_text.str.findall(last_pattern)[last_pattern_filter].index

In [ ]:
corrected_text[indexes].iloc[1]

In [ ]:
from src.features.text.transformers.encoding_corrector import SpaceReplacement

replacer = SpaceReplacement()
final_text = replacer.fit_transform(corrected_text)

In [ ]:
final_text.str.count(pattern1).sum(), final_text.str.count(pattern2).sum(), final_text.str.count(pattern3).sum(),  final_text.str.count(pattern4).sum()
final_text.str.findall(last_pattern)[last_pattern_filter].to_list()
